In [1]:
import os

os.chdir("../")
os.getcwd()

'c:\\Users\\HP\\Desktop\\GenHack\\Genhack_competition'

In [2]:
# imports
import requests
from bs4 import BeautifulSoup

In [3]:
# params
URL = "https://competitions.outcoder.ai/leaderboards"

In [4]:
# fetch page and build soup
res = requests.get(URL)
document = BeautifulSoup(res.text)

In [5]:
# locate cols of interset
col_anderson, col_kendall = document.find_all(
    "div",
    class_="w-col w-col-6 w-col-small-6 w-col-tiny-6"
)

In [6]:
# data structure
{
    "team_name": {
        "anderson": {
            "board": 0,
            "z-score": 0,
            "real": 0
        },
        "kendal_board": {
            "board": 0,
            "z-score": 0,
            "real": 0
        },
    },
    # ...
}

{'team_name': {'anderson': {'board': 0, 'z-score': 0, 'real': 0},
  'kendal_board': {'board': 0, 'z-score': 0, 'real': 0}}}

In [7]:
# where to store scores
leader_board = {}

In [8]:
# split teams sections
# for anderson
teams_sections = col_anderson.find_all("div", class_="w-dyn-item")

for team in teams_sections:
    # explode content of team
    team_content = team.find_all("div", class_="w-col w-col-3")

    # select name and score
    team_score = team_content[-1].text
    team_name = team_content[-2].text

    # save score 
    # add 0.001 to avoid 0 scores
    leader_board[team_name] = { 
        "anderson": {
            "board": float(team_score) + 0.001,
            "z-score": 0,
            "real": 0
        }
    }

In [9]:
# do the same for kendall
teams_sections = col_kendall.find_all("div", class_="w-dyn-item")

for team in teams_sections:
    # select name and score
    team_score = team.find_all("div", class_="w-col w-col-3")[-1].text
    team_name = team.find("div", class_="column-3 w-col w-col-3").text

    # save score
    # add 0.001 to avoid 0 scores
    leader_board[team_name]["kendall"] = {
            "board": float(team_score) + 0.001,
            "z-score": 0,
            "real": 0
    }

In [10]:
leader_board

{'Winx': {'anderson': {'board': 647.5409999999999, 'z-score': 0, 'real': 0},
  'kendall': {'board': 747.761, 'z-score': 0, 'real': 0}},
 '5 Layers of Stonks': {'anderson': {'board': 647.3109999999999,
   'z-score': 0,
   'real': 0},
  'kendall': {'board': 744.1809999999999, 'z-score': 0, 'real': 0}},
 'DL Manjikai': {'anderson': {'board': 647.261, 'z-score': 0, 'real': 0},
  'kendall': {'board': 742.081, 'z-score': 0, 'real': 0}},
 'Terran': {'anderson': {'board': 647.031, 'z-score': 0, 'real': 0},
  'kendall': {'board': 755.011, 'z-score': 0, 'real': 0}},
 'Lbitrik': {'anderson': {'board': 646.761, 'z-score': 0, 'real': 0},
  'kendall': {'board': 735.131, 'z-score': 0, 'real': 0}},
 'Medieval Mates': {'anderson': {'board': 646.6709999999999,
   'z-score': 0,
   'real': 0},
  'kendall': {'board': 716.881, 'z-score': 0, 'real': 0}},
 'Random Guild': {'anderson': {'board': 645.991, 'z-score': 0, 'real': 0},
  'kendall': {'board': 546.191, 'z-score': 0, 'real': 0}},
 'ECES': {'anderson': 

In [11]:
# compute z-scores
from scipy.stats import norm


# loop over teams
for team_name, team_scores in leader_board.items():
    # loop over scores
    for metric_name, metric_scores in team_scores.items():
        board_score = metric_scores["board"]

        leader_board[team_name][metric_name]["z-score"] = norm.ppf(board_score / 1000)

In [12]:
leader_board

{'Winx': {'anderson': {'board': 647.5409999999999,
   'z-score': 0.37869010787150503,
   'real': 0},
  'kendall': {'board': 747.761, 'z-score': 0.6674605507048509, 'real': 0}},
 '5 Layers of Stonks': {'anderson': {'board': 647.3109999999999,
   'z-score': 0.37807079924354703,
   'real': 0},
  'kendall': {'board': 744.1809999999999,
   'z-score': 0.6562893012562411,
   'real': 0}},
 'DL Manjikai': {'anderson': {'board': 647.261,
   'z-score': 0.377936186127173,
   'real': 0},
  'kendall': {'board': 742.081, 'z-score': 0.6497743340973121, 'real': 0}},
 'Terran': {'anderson': {'board': 647.031,
   'z-score': 0.3773170539471422,
   'real': 0},
  'kendall': {'board': 755.011, 'z-score': 0.6903438155675782, 'real': 0}},
 'Lbitrik': {'anderson': {'board': 646.761,
   'z-score': 0.376590431092823,
   'real': 0},
  'kendall': {'board': 735.131, 'z-score': 0.6284060115218624, 'real': 0}},
 'Medieval Mates': {'anderson': {'board': 646.6709999999999,
   'z-score': 0.37634826767194224,
   'real': 0

In [13]:
from py_scripts.utils import save_dict

save_dict(
    leader_board,
    "reverse_scores/leaderboard"
)